In [93]:
class GameTree:
    def __init__(self, value=None, children=None):
        self.value = value
        self.children = children if children is not None else []

    def add_child(self, child):
        self.children.append(child)

    def is_leaf(self):
        return len(self.children) == 0

In [94]:
# Displaying the structure of the tree
def display_tree(node, level=0):
    indent = " " * (4 * level)

    if node.value[1] is not None:
        print(f"{indent}{node.value[1]}")
    else: 
        print(f"{indent}{node.value[0]}")
    for child in node.children:
        display_tree(child, level + 1)

In [95]:
# depth 0
x_01 = GameTree(['x_01', None])

# depth 1
x_11 = GameTree(['x_11', 5])
x_01.add_child(x_11)

x_12 = GameTree(['x_12', None])
x_01.add_child(x_12)

# depth 2
x_21 = GameTree(['x_21', None])
x_12.add_child(x_21)

x_22 = GameTree(['x_22', None])
x_12.add_child(x_22)

# depth 3
x_31 = GameTree(['x_31', 1])
x_21.add_child(x_31)

x_32 = GameTree(['x_32', None])
x_21.add_child(x_32)

x_33 = GameTree(['x_33', 5])
x_22.add_child(x_33)

x_34 = GameTree(['x_34', None])
x_22.add_child(x_34)

# depth 4

x_41 = GameTree(['x_41', 4]) 
x_32.add_child(x_41)

x_42 = GameTree(['x_42', 2])
x_32.add_child(x_42)

x_43 = GameTree(['x_43', 4])
x_34.add_child(x_43)

x_44 = GameTree(['x_44', 3])
x_34.add_child(x_44)


# Display the tree
display_tree(x_01)


x_01
    5
    x_12
        x_21
            1
            x_32
                4
                2
        x_22
            5
            x_34
                4
                3


In [96]:
def minimax(node, depth, is_maximizing_player):
    # Print the current node being considered

    # Base case: if the current node is a leaf node
    if node.is_leaf():
        return node.value[1]

    # Recursive case: if the current node is a variable node
    if is_maximizing_player:
        best_value = float('-inf')
        for child in node.children:
            value = minimax(child, depth + 1, False)
            best_value = max(best_value, value)
        return best_value
    else:
        best_value = float('inf')
        for child in node.children:
            value = minimax(child, depth + 1, True)
            best_value = min(best_value, value)
        return best_value

# Applying the Minimax algorithm to the root of the game tree
minimax_value = minimax(x_01, 0, True)
print(f"\nThe value of the root node (x_01) after applying Minimax is: {minimax_value}")



The value of the root node (x_01) after applying Minimax is: 5


In [97]:
def alpha_beta_pruning(node, depth, alpha, beta, is_maximizing_player):

    # Base case: if the current node is a leaf node
    if node.is_leaf():
        return node.value[1]

    # Recursive case: if the current node is a variable node
    if is_maximizing_player:
        best_value = float('-inf')
        for child in node.children:
            value = alpha_beta_pruning(child, depth + 1,  alpha, beta, False)
            best_value = max(best_value, value)
            alpha = max(alpha, value)

            print(f'current node:{node.value[0]}, after returning from child: {child.value[0]}: (alpha:{alpha}, beta:{beta}) ')
            if beta <= alpha:
                print(f'\tPruning right subtree at node: {node.value[0]}:{best_value}')
                break
        return best_value
    else:
        best_value = float('inf')
        for child in node.children:
            value = alpha_beta_pruning(child, depth + 1,  alpha, beta, True)
            best_value = min(best_value, value)
            beta = min(beta, value)
            print(f'current node:{node.value[0]}, after returning from child: {child.value[0]}: (alpha:{alpha}, beta:{beta}) ')
            if beta <= alpha:
                print(f'\tPruning right subtree at node: {node.value[0]}:{best_value}')
                break

        return best_value


alpha_beta_pruning_value = alpha_beta_pruning(x_01, 0, float('-inf'), float('inf'), True)
print(f"\nThe value of the root node (x_01) after applying Minimax is: {alpha_beta_pruning_value}")


current node:x_01, after returning from child: x_11: (alpha:5, beta:inf) 
current node:x_21, after returning from child: x_31: (alpha:5, beta:inf) 
current node:x_32, after returning from child: x_41: (alpha:5, beta:4) 
	Pruning right subtree at node: x_32:4
current node:x_21, after returning from child: x_32: (alpha:5, beta:inf) 
current node:x_12, after returning from child: x_21: (alpha:5, beta:4) 
	Pruning right subtree at node: x_12:4
current node:x_01, after returning from child: x_12: (alpha:5, beta:inf) 

The value of the root node (x_01) after applying Minimax is: 5
